# **FROZEN LAKE**


## **Set-up environment**


In [ ]:
import numpy as np

In [156]:
ENV_ROWs = 4
ENV_COLs = 4
ACTIONS = ['left', 'down', 'right', 'up']
HOLES = [(1, 1), (1, 3), (2, 3), (3, 0)]
START = (0, 0)
GOAL = (3, 3)

In [157]:
# Create rewards matrix
REWARDS = np.full((ENV_ROWs, ENV_COLs), 0.)
REWARDS[ENV_ROWs-1, ENV_COLs-1] = 1.
for row in REWARDS:
    print(row)

[0. 0. 0. 0.]
[0. 0. 0. 0.]
[0. 0. 0. 0.]
[0. 0. 0. 1.]


## **Create Q-table**


In [158]:
# CREATE q-table
Q_TABLE = np.full((ENV_ROWs*ENV_COLs, len(ACTIONS)), 0.0)
Q_TABLE

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

## **Define some helper functions**


In [159]:
def is_terminal_state(current_row_index, current_column_index):
    """
    Check if the current state is a terminal state.
    Terminal states are the goal and the "holes" locations.
    """
    position = (current_row_index, current_column_index)
    if position == GOAL or position in HOLES:
        return True
    else:
        return False


def get_next_action(current_row_index, current_column_index, epsilon):
    """
    Given the current state, select the next action to take based on the epsilon-greedy policy.
    """
    # if a randomly chosen value between 0 and 1 is less than epsilon,
    # then choose the most promising value from the Q-table for this state.
    # else choose a random action.
    if np.random.random() < epsilon:
        return np.argmax(Q_TABLE[4*current_row_index+current_column_index, :])
    else:
        return np.random.randint(4)


def get_next_location(current_row_index, current_column_index, action_index):
    """
    Return the next location based on the action taken and the current location.
    """
    new_row_index = current_row_index
    new_column_index = current_column_index
    if ACTIONS[action_index] == 'up' and current_row_index > 0:
        new_row_index -= 1
    elif ACTIONS[action_index] == 'right' and current_column_index < ENV_COLs - 1:
        new_column_index += 1
    elif ACTIONS[action_index] == 'down' and current_row_index < ENV_ROWs - 1:
        new_row_index += 1
    elif ACTIONS[action_index] == 'left' and current_column_index > 0:
        new_column_index -= 1
    return new_row_index, new_column_index


def get_path_to_goal(start_row_index, start_column_index):
    """
    Return the path to the goal using the greedy approach.
    """
    # return immediately if this is an invalid starting location
    if is_terminal_state(start_row_index, start_column_index):
        return []
    else:
        current_row_index, current_column_index = start_row_index, start_column_index
        path_to_goal = []
        path_to_goal.append([current_row_index, current_column_index])
        # continue moving along the path until we reach the goal (i.e., the item packaging location)
        while not is_terminal_state(current_row_index, current_column_index):
            # get the best action to take
            best_action_index = np.argmax(
                Q_TABLE[4*current_row_index+current_column_index, :])
            # move to the next location on the path, and add the new location to the list
            current_row_index, current_column_index = get_next_location(
                current_row_index, current_column_index, best_action_index)
            path_to_goal.append([current_row_index, current_column_index])
        return path_to_goal

## **Train agent**


### **Define hyperparameters**


In [161]:
total_episodes = 10000
max_steps = 99

learning_rate = 0.7

# discounting rate
gamma = 0.95

# exploration rate
max_epsilon = 1.0
min_epsilon = 0.05
decay_rate = 0.0005

### **Train using Q-learning algorithm**


In [162]:
for episode in range(total_episodes):

    # change epsilon for this episode
    epsilon = min_epsilon + (max_epsilon - min_epsilon) * \
        np.exp(-decay_rate*episode)

    # get the starting location for this episode
    row_index, column_index = START

    for step in range(max_steps):
        action_index = get_next_action(row_index, column_index, epsilon)

        old_row_index, old_column_index = row_index, column_index
        row_index, column_index = get_next_location(
            row_index, column_index, action_index)

        reward = REWARDS[row_index, column_index]
        old_q_value = Q_TABLE[4*old_row_index + old_column_index, action_index]
        temporal_difference = reward + \
            (gamma * np.max(Q_TABLE[4*row_index +
             column_index, :])) - old_q_value

        # update the Q-value for the previous state and action pair
        new_q_value = old_q_value + (learning_rate * temporal_difference)
        Q_TABLE[4*old_row_index + old_column_index, action_index] = new_q_value

        done = is_terminal_state(row_index, column_index)
        if done == True:
            break

print('Training complete!')

Training complete!


## **View Q-table results**


In [163]:
import pandas as pd
q_df = pd.DataFrame(np.round(Q_TABLE, 3), columns=ACTIONS)
q_df

,left,down,right,up
0,0.735,0.774,0.774,0.735
1,0.735,0.000,0.815,0.774
2,0.774,0.857,0.774,0.815
3,0.815,0.000,0.774,0.774
4,0.774,0.815,0.000,0.735
5,0.000,0.000,0.000,0.000
6,0.000,0.902,0.000,0.815
7,0.000,0.000,0.000,0.000
8,0.815,0.000,0.857,0.774
9,0.815,0.902,0.902,0.000


In [164]:
print('Shortest path from start to goal:')
print(get_path_to_goal(0, 0))

Shortest path from start to goal:
[[0, 0], [1, 0], [2, 0], [2, 1], [3, 1], [3, 2], [3, 3]]
